# Neighbor Discovery Simulation
The following code demonstrates the usage of the BLE neighbor discovery simulation presented in "". 

In [ ]:
# Imports from Python Library
from DiscoverySimulation import DiscoverySimulation, calc_mean_for_paramset

In [ ]:
# Specify BLE Discovery Parameters 
T = 5.12
Ts = 3
Ta = 4

# Specify Paramset as Triple (T, Ts, Ta)
paramset = (T, Ts, Ta)

# Run Simulation for Paramset
result = calc_mean_for_paramset(paramset=paramset, samples=10000, rngloss=0.85)

# Result is a tuple of (paramset, (mean, stdDev), individualResults), unpack it
paramset, (mean, stdDev), individualResults = result

print("Paramset", paramset, "resulted in", mean, "seconds mean discovery latency")

# Advanced usage
the calc_mean_for_paramset() function is a helper for running the simulation with a default set of other parameters (slotsize for simulation, advertisement length, advertisement random delay, ...). These usually provide a good starting point.
However if more fine grained control is needed, the DiscoverySimulation class allows for more advanced setups. Every time based parameter given to the class is a number of slots. Every time based return value is a number of slots. You have to do the conversion by yourself.

In [ ]:
SECONDS_PER_SLOT = 0.625 * 10**(-6)

T = 5.12
Ts = 3
Ta = 4
paramset = (T, Ts, Ta)

TSlots = T/SECONDS_PER_SLOT
TsSlots = Ts/SECONDS_PER_SLOT
TaSlots = Ta/SECONDS_PER_SLOT

advSize = 20 # Advertisement Size in Bytes

# 20 Bytes / 1MBit/s
advLength = (advSize*8) / 1000000 # Advertisement PDU Length
advLengthSlots = advLength/SECONDS_PER_SLOT # Advertisement PDU Length in Slots
bLengthSlots = (0.01 - advLength)/SECONDS_PER_SLOT # Advertisement delay in Slots

randomslots = 0.01/SECONDS_PER_SLOT # Random Advertisement Interval in Slots

rngLoss = 0.85 # Loss of Advertisement matches

samples = 10000 # Sample counf for calculating mean values

sim = DiscoverySimulation(TSlots, TsSlots, TaSlots, advLengthSlots, bLengthSlots, randomslots, rngLoss)
meanSlots, stdDevSlots, individualResults = sim.mean(samples=samples)

mean = meanSlots * SECONDS_PER_SLOT
stdDev = stdDevSlots * SECONDS_PER_SLOT

print("Paramset", paramset, "resulted in", mean, "seconds mean discovery latency")

## Iterating over parameter space
Sometimes it is necessary to iterate the simulation over a whole range of parameters. This can be made by using python generators and a process pool for faster execution.
It is advised to use a downsampling factor if iterating over a large parameter space.

In [ ]:
from multiprocessing import Pool

factor = 5000
def genrange(minarg, maxarg, step):
    """
    Generator for range 
    """
    k = minarg
    while k <= maxarg:
        yield k
        k += step

def parameterGenerator():
    """
    Generator for all valid parameter set tuples (P in paper)
    """
    MS = 10**(-3)

    for T in genrange(20*MS, 10.24, 0.625*MS * factor):  # Scanning Interval
        for Ts in genrange(20*MS, 10.24, 0.625*MS * factor):                      # Scanning Window
            for Ta in genrange(20*MS, 10.24, 0.625*MS * factor):
                if Ts <= T:
                    yield (T, Ts, Ta)

In [ ]:
with Pool(16) as p:
    results = p.imap_unordered(calc_mean_for_paramset, parameterGenerator())

    for paramset, stats, individualResults in results:
        (T, Ts, Ta) = paramset
        (meanD, stdDev) = stats
        
        print("Paramset", paramset, "resulted in", meanD, "seconds mean discovery latency")